## steps
1. ✅Understand the whole picture
   - Quantify final objectives ?
   - What is the current solution (if any) ? Downsides ?
2. ✅Get data
3. ✅explore, visualize data => **insight**
4. ✅prepare data for machine learning algo
5. select and train a model <- 
6. fine-tune model 
   - Mesure the errors made by the model, with **RMSE** or **MAE**
7. present solution
8.  launch, monitor, maintain system

In [1]:
import pandas as pd
housing_prepared = pd.read_feather('housing-data-prepared.arrow')

In [2]:
housing_prepared

,bedrooms__ratio,rooms_per_house__ratio,people_per_house__ratio,log__total_bedrooms,log__total_rooms,log__population,log__households,log__median_income,geo__Cluster_0_similarity,geo__Cluster_1_similarity,...,geo__Cluster_7_similarity,geo__Cluster_8_similarity,geo__Cluster_9_similarity,cat__ocean_proximity_<1H OCEAN,cat__ocean_proximity_INLAND,cat__ocean_proximity_ISLAND,cat__ocean_proximity_NEAR BAY,cat__ocean_proximity_NEAR OCEAN,remainder__housing_median_age,remainder__income_cat
0,-0.121122,0.017395,0.006223,0.873403,0.852032,0.938249,0.797843,-0.998832,3.391002e-16,4.706064e-01,...,8.217042e-01,1.493399e-07,1.579689e-01,0.0,1.0,0.0,0.0,0.0,0.027564,-0.954687
1,-0.810867,0.569256,-0.040811,0.970407,1.277433,0.796538,0.913214,1.277742,4.197435e-01,1.850749e-16,...,1.272655e-22,6.521017e-05,7.678546e-18,0.0,0.0,0.0,0.0,1.0,-1.722018,1.890078
2,-0.338273,-0.018024,-0.075371,-0.430559,-0.317644,-0.702499,-0.375177,-0.403837,1.810043e-02,6.391756e-05,...,2.191546e-08,5.466398e-01,6.801739e-06,0.0,1.0,0.0,0.0,0.0,1.220460,-0.954687
3,0.961205,-0.595140,-0.106803,0.281063,-0.118663,-0.299288,0.277558,-0.947769,1.937594e-01,3.147969e-18,...,9.457060e-25,9.793789e-06,1.564997e-19,0.0,0.0,0.0,0.0,1.0,-0.370069,-0.954687
4,-0.474513,0.251241,0.006109,0.583337,0.730040,0.671139,0.528397,0.547635,2.991692e-01,2.354275e-08,...,5.612523e-13,2.281453e-01,4.312614e-09,1.0,0.0,0.0,0.0,0.0,-0.131489,-0.006432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16507,-0.443846,0.466353,-0.092580,1.473719,1.579483,0.803919,1.276380,0.811447,3.213731e-01,3.071443e-17,...,1.680733e-23,2.291349e-05,1.005121e-18,1.0,0.0,0.0,0.0,0.0,-1.165333,0.941823
16508,-0.516065,0.050362,0.023024,1.672897,1.807457,1.964650,1.753694,-0.449539,5.028941e-16,4.950336e-01,...,8.256769e-01,1.842162e-07,1.557476e-01,0.0,1.0,0.0,0.0,0.0,-1.085806,-0.954687
16509,0.322869,-0.509240,-0.037436,-1.293050,-1.427119,-1.212202,-1.137635,-0.189525,1.008715e-20,3.273126e-02,...,9.877901e-02,6.409134e-10,1.298829e-01,1.0,0.0,0.0,0.0,0.0,1.538566,-0.006432
16510,-0.457023,0.328149,-0.059156,0.434515,0.577261,0.102799,0.327708,0.400730,3.936388e-20,4.081377e-02,...,9.069403e-02,1.753811e-09,1.796418e-01,1.0,0.0,0.0,0.0,0.0,-1.165333,-0.006432
